In [273]:
import numpy as np
import pandas as pd
import glob
import re
from datetime import datetime
from sklearn.preprocessing import LabelEncoder


In [274]:
def is_top_model(x, models):
    if models.count(x)==0:
        x = "other"
    return x


## TEST DATA


In [275]:
# kaggle only:

'''
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

df_test = pd.read_csv('../input/sf-dst-car-price-prediction/test.csv')
df_train = pd.read_csv('../input/parsed-auto/train.csv')
df_sub = pd.read_csv('../input/sf-dst-car-price-prediction/sample_submission.csv')

'''


#local:
#path = "/"

# remote:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/My Drive/module_6/"


df_test = pd.read_csv(path + 'test.csv')
df_sub = pd.read_csv(path + 'sample_submission.csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## COLLECT TRAIN DATA:


In [276]:
# collect data from files in CONTENT folder:
filenames = glob.glob(path + "CONTENT/*content.csv")
lst_df_train = []
for filename in filenames:
    df_cur = pd.read_csv(filename, header=None)
    lst_df_train.append(df_cur)
df_train =  pd.concat(lst_df_train, axis=0, ignore_index=True)    

# get rid of duplicates and small check:   
df_train.drop_duplicates(inplace=True)
df_train.shape


(35265, 23)

## SAVE DF_TRAIN

In [277]:
# save to file:
df_train.to_csv(path + "train.csv", index=False)


In [278]:
df_train.sample(3)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
754,SKODA,Skoda Kodiaq I,внедорожник 5 дв.,5,2016,2018,серый,1.4 LTR,150 N12,бензин,https://autoru.naydex.net/Lr84PuH52/0ca95dIpb/...,105 000 км,1.4 AMT,ALLROAD_5_DOORS ROBOT 1.4,роботизированная,2 владельца,Оригинал,передний,Левый,Не требует ремонта,Растаможен,https://auto.ru/cars/used/sale/skoda/kodiaq/11...,1 849 000 ₽
5338,AUDI,Audi A6 III (C6) Рестайлинг,седан,4,2008,2010,чёрный,3.0 LTR,239 N12,дизель,https://avatars.mds.yandex.net/get-autoru-vos/...,244 000 км,3.0 AT,SEDAN AUTOMATIC 3.0,автоматическая,3 или более,Оригинал,полный,Левый,Не требует ремонта,Растаможен,https://auto.ru/cars/used/sale/audi/a6/1105443...,1 210 000 ₽
6780,HONDA,Honda Odyssey (North America) V,минивэн,5,2017,2018,серый,3.5 LTR,280 N12,бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,29 800 км,3.5 AT,MINIVAN AUTOMATIC 3.5,автоматическая,1 владелец,Оригинал,передний,Левый,Не требует ремонта,Растаможен,https://auto.ru/cars/used/sale/honda/odyssey_n...,2 495 000 ₽


In [279]:
df_test.head()


,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,74000,2013,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.2 AMT (105 л.с.),5,1603226273,RUB,2014,1100575026,"{""id"":""10373605"",""displacement"":1197,""engine_t...",LIFTBACK ROBOT 1.2,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,60563,2017,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.6 MT (110 л.с.),5,1603226277,RUB,2017,1100549428,"{""id"":""20913311"",""displacement"":1598,""engine_t...",LIFTBACK MECHANICAL 1.6,механическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
2,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/superb/11...,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,88000,2013,"{""code"":""SUPERB"",""name"":""Superb"",""ru_name"":""Су...",SUPERB,DSG 1.8 AMT (152 л.с.),5,1603226280,RUB,2014,1100658222,"{""id"":""20026323"",""nameplate"":""DSG"",""displaceme...",LIFTBACK ROBOT 1.8,роботизированная,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
3,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,коричневый,"{""id"":""20803582"",""name"":""Ambition"",""available_...",КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6 LTR,110 N12,"{""cruise-control"":true,""roller-blind-for-rear-...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,95000,2013,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.6 AT (110 л.с.),5,1603226284,RUB,2014,1100937408,"{""id"":""20105521"",""displacement"":1598,""engine_t...",LIFTBACK AUTOMATIC 1.6,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
4,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,белый,NaN,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8 LTR,152 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,58536,2008,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.8 AT (152 л.с.),5,1603226288,RUB,2012,1101037972,"{""id"":""4561004"",""displacement"":1798,""engine_ty...",LIFTBACK AUTOMATIC 1.8,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен


In [280]:
df_sub.head()

,sell_id,price
0,1100575026,0
1,1100549428,0
2,1100658222,0
3,1100937408,0
4,1101037972,0


## COLUMNS:


- compare columns in train and test datasets, rename where necessary and drop from train if needed
- mark test and train data
- join datasets


In [281]:
# take a look at test columns:
test_columns = df_test.columns
test_columns


Index(['bodyType', 'brand', 'car_url', 'color', 'complectation_dict',
       'description', 'engineDisplacement', 'enginePower', 'equipment_dict',
       'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name',
       'name', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency',
       'productionDate', 'sell_id', 'super_gen', 'vehicleConfiguration',
       'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС',
       'Привод', 'Руль', 'Состояние', 'Таможня'],
      dtype='object')

In [282]:
# take a look at train columns:
train_columns = df_train.columns
train_columns


Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22],
           dtype='int64')

In [283]:
# ooops... take another look at data to figure out how to name them:
df_train.head().T


,0,1,2,3,4
0,SKODA,SKODA,SKODA,SKODA,SKODA
1,Skoda Fabia II,Skoda Fabia I,Skoda Octavia III (A7) Рестайлинг,Skoda Fabia RS II,Skoda Fabia II Рестайлинг
2,универсал 5 дв.,универсал 5 дв.,лифтбек,хэтчбек 5 дв.,хэтчбек 5 дв.
3,5,5,5,5,5
4,2007,1999,2017,2010,2010
5,2009,2001,2017,2012,2011
6,чёрный,серебристый,белый,синий,чёрный
7,1.4 LTR,1.4 LTR,1.6 LTR,1.4 LTR,1.2 LTR
8,86 N12,68 N12,110 N12,180 N12,70 N12
9,бензин,бензин,бензин,бензин,бензин


In [284]:
# rename train columns:
train_columns = train_columns.map({0 : 'brand', 
                                   1 : 'model_name', 
                                   2 : 'bodyType',
                                   3 : 'numberOfDoors',
                                   4 : 'modelDate',
                                   5 : 'productionDate',
                                   6 : 'color',
                                   7 : 'engineDisplacement',
                                   8 : 'enginePower',
                                   9 : 'fuelType',
                                   10 : 'image',
                                   11 : 'mileage',
                                   12 : 'name',
                                   13 : 'vehicleConfiguration',
                                   14 : 'vehicleTransmission',
                                   15 : 'Владельцы',
                                   16 : 'ПТС',
                                   17 : 'Привод',
                                   18 : 'Руль',
                                   19 : 'Состояние',
                                   20 : 'Таможня', 
                                   21 : 'car_url',
                                   22 : 'price'})
df_train.columns = train_columns


In [285]:
# take a look if all good now:
df_train.sample().T


,17292
brand,NISSAN
model_name,Nissan Qashqai II Рестайлинг
bodyType,внедорожник 5 дв.
numberOfDoors,5
modelDate,2017
productionDate,2020
color,серый
engineDisplacement,2.0 LTR
enginePower,144 N12
fuelType,бензин


In [286]:
# find columns in test not being presented in train:
cols_to_drop_from_test = list(set(test_columns) - set(train_columns))
cols_to_drop_from_test


['priceCurrency',
 'equipment_dict',
 'model_info',
 'super_gen',
 'description',
 'Владение',
 'vendor',
 'sell_id',
 'parsing_unixtime',
 'complectation_dict']

In [287]:
# hmmmm ))) interesting, how old test dataset is:
datetime.fromtimestamp(df_test["parsing_unixtime"].min()).strftime('%Y-%b-%d'),\
datetime.fromtimestamp(df_test["parsing_unixtime"].max()).strftime('%Y-%b-%d')


('2020-Oct-19', '2020-Oct-26')

Данным ровно год. Цены выросли. По данным статистики, на автомобили - на 15% Возможно, следует промасштабировать цену...

In [288]:
# priceCurrency - hopefulyy, the same everywhere:
df_test.priceCurrency.value_counts()


RUB    34686
Name: priceCurrency, dtype: int64

OK, as in train dataset all prices are in RUB 

In [289]:
# check if we've got new columns during parsing:
new_columns = list(set(train_columns) - set(test_columns))
new_columns


['price']

that's ok, we are not supposed to know the price in test dataset


In [290]:
# compare main values in test and train to make sure we have good data:
df_test.brand.value_counts(), "==================", df_train.brand.value_counts()


(BMW           4473
 VOLKSWAGEN    4404
 NISSAN        4393
 MERCEDES      4180
 TOYOTA        3913
 AUDI          3421
 MITSUBISHI    2843
 SKODA         2741
 VOLVO         1463
 HONDA         1150
 INFINITI       871
 LEXUS          834
 Name: brand, dtype: int64, '==================', AUDI          3383
 SKODA         3383
 HONDA         3377
 BMW           3162
 NISSAN        3160
 MERCEDES      3083
 MITSUBISHI    3065
 TOYOTA        2992
 VOLKSWAGEN    2980
 LEXUS         2926
 VOLVO         2411
 INFINITI      1343
 Name: brand, dtype: int64)

There are only 12 brands represented in test dataset , for now therefore no need to process others

In [291]:
# join datasets marking them for future split:
df_test['price'] = 0
df_test['test_train'] = "test"
df_train['test_train'] = "train"
data = df_train.append(df_test, sort=False).reset_index(drop=True) # объединяем
data.shape


(69951, 34)

In [292]:
# remove columns we dot't have in train:
data.drop(cols_to_drop_from_test, axis=1, inplace=True)
data.shape


(69951, 24)

In [293]:
# rename columns to make their names more commfortable to use:
col_replacement = {'bodyType' : 'body_type', 
                   'numberOfDoors': 'doors',
                  'modelDate':'model_date',
                  'productionDate': 'prod_date',
                  'engineDisplacement':'engine_V',
                  'enginePower' : 'power',
                  'fuelType': 'fuel',
                  'vehicleConfiguration' : 'config',
                  'vehicleTransmission' : 'transmission',
                  'Владельцы' : 'owners',
                   'ПТС' : 'pass',
                   'Привод' : 'drive',
                   'Руль' : 'wheel',
                   'Состояние' : 'state',
                   'Таможня' : 'custom'
                  }
data.rename(columns=col_replacement, inplace=True)
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69951 entries, 0 to 69950
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         69951 non-null  object
 1   model_name    69951 non-null  object
 2   body_type     69951 non-null  object
 3   doors         69951 non-null  int64 
 4   model_date    69951 non-null  int64 
 5   prod_date     69951 non-null  int64 
 6   color         69951 non-null  object
 7   engine_V      69951 non-null  object
 8   power         69951 non-null  object
 9   fuel          69951 non-null  object
 10  image         69951 non-null  object
 11  mileage       69951 non-null  object
 12  name          69951 non-null  object
 13  config        69951 non-null  object
 14  transmission  69951 non-null  object
 15  owners        69951 non-null  object
 16  pass          69950 non-null  object
 17  drive         69951 non-null  object
 18  wheel         69951 non-null  object
 19  stat

## VALUE CLEANUP


- process columns manually one by one
- split column names into 3 lists: categorian, numeric and binary
- add new columns
- drop some columns


In [294]:
# lists to sort out columns for future processing:
cat_columns = []
num_columns = []
bin_columns = []


In [295]:
# when necesssary turn this on:
# pd.set_option('display.max_rows', None)


### 0. brand

In [296]:
# how many cars of each brand do we have:
data.brand.value_counts()


BMW           7635
NISSAN        7553
VOLKSWAGEN    7384
MERCEDES      7263
TOYOTA        6905
AUDI          6804
SKODA         6124
MITSUBISHI    5908
HONDA         4527
VOLVO         3874
LEXUS         3760
INFINITI      2214
Name: brand, dtype: int64

In [297]:
cat_columns.append("brand")


### 1. model_name

In [298]:
data.model_name.value_counts()


OCTAVIA                                    1418
5ER                                        1026
LANCER                                      855
E_KLASSE                                    809
PASSAT                                      782
                                           ... 
Mercedes-Benz S-Класс 600 Long V (W221)       1
Mercedes-Benz C-Класс 300 IV (W205)           1
Mercedes-Benz Vito 109 CDI L1 II (W639)       1
BMW Z4 35i II (E89)                           1
100_SERIES                                    1
Name: model_name, Length: 2861, dtype: int64

In [299]:
models_to_numbers = dict(data.model_name.value_counts())
models_to_numbers


{'OCTAVIA': 1418,
 '5ER': 1026,
 'LANCER': 855,
 'E_KLASSE': 809,
 'PASSAT': 782,
 'A6': 757,
 '3ER': 752,
 'POLO': 724,
 'CAMRY': 709,
 'X5': 675,
 'X_TRAIL': 654,
 'A4': 626,
 'QASHQAI': 609,
 'S_KLASSE': 549,
 'OUTLANDER': 544,
 'LAND_CRUISER': 514,
 'ALMERA': 502,
 'TIGUAN': 491,
 'Volkswagen Polo V Рестайлинг': 473,
 'TOUAREG': 469,
 'COROLLA': 456,
 'RAV_4': 447,
 'C_KLASSE': 428,
 'GOLF': 424,
 'Skoda Octavia II (A5) Рестайлинг': 400,
 'LAND_CRUISER_PRADO': 387,
 'TEANA': 374,
 'RAPID': 370,
 'Skoda Octavia III (A7) Рестайлинг': 361,
 'Skoda Octavia III (A7)': 356,
 'M_KLASSE': 343,
 '7ER': 343,
 'XC90': 340,
 'JETTA': 324,
 'X6': 317,
 'CIVIC': 316,
 'Volkswagen Tiguan I Рестайлинг': 311,
 'PAJERO': 306,
 'Nissan Qashqai II': 305,
 'Q7': 302,
 'FX': 297,
 'X3': 283,
 'Volvo XC90 I Рестайлинг': 277,
 'Nissan X-Trail III': 272,
 'JUKE': 268,
 'Volkswagen Tiguan II': 267,
 'Skoda Rapid I Рестайлинг': 262,
 'ACCORD': 260,
 'SUPERB': 248,
 'Mitsubishi Outlander III Рестайлинг 2': 24

In [300]:
top_models = []
for model in models_to_numbers:
    if models_to_numbers[model]>=30:
        top_models.append(model)
top_models


['OCTAVIA',
 '5ER',
 'LANCER',
 'E_KLASSE',
 'PASSAT',
 'A6',
 '3ER',
 'POLO',
 'CAMRY',
 'X5',
 'X_TRAIL',
 'A4',
 'QASHQAI',
 'S_KLASSE',
 'OUTLANDER',
 'LAND_CRUISER',
 'ALMERA',
 'TIGUAN',
 'Volkswagen Polo V Рестайлинг',
 'TOUAREG',
 'COROLLA',
 'RAV_4',
 'C_KLASSE',
 'GOLF',
 'Skoda Octavia II (A5) Рестайлинг',
 'LAND_CRUISER_PRADO',
 'TEANA',
 'RAPID',
 'Skoda Octavia III (A7) Рестайлинг',
 'Skoda Octavia III (A7)',
 'M_KLASSE',
 '7ER',
 'XC90',
 'JETTA',
 'X6',
 'CIVIC',
 'Volkswagen Tiguan I Рестайлинг',
 'PAJERO',
 'Nissan Qashqai II',
 'Q7',
 'FX',
 'X3',
 'Volvo XC90 I Рестайлинг',
 'Nissan X-Trail III',
 'JUKE',
 'Volkswagen Tiguan II',
 'Skoda Rapid I Рестайлинг',
 'ACCORD',
 'SUPERB',
 'Mitsubishi Outlander III Рестайлинг 2',
 'S60',
 'Q5',
 'FABIA',
 'Volkswagen Polo V',
 'Skoda Fabia II Рестайлинг',
 'Skoda Rapid I',
 'Mitsubishi Outlander III',
 'YETI',
 'MURANO',
 'CR_V',
 'RX',
 'GL_KLASSE',
 'A3',
 'Honda Civic VIII',
 'Skoda Kodiaq I',
 '1ER',
 'PRIMERA',
 'X1',
 

In [301]:
# reduce number of categories:
# if number of records for a model less than 30, replace model_name with "other" ...
data.model_name = data.model_name.apply(lambda x: is_top_model(x, top_models))


In [302]:
# sort out:
cat_columns.append("model_name")


### 2. body_type

In [303]:
data.body_type.value_counts()


внедорожник 5 дв.          27985
седан                      23726
лифтбек                     4919
хэтчбек 5 дв.               4270
универсал 5 дв.             2526
минивэн                     1684
купе                        1460
компактвэн                  1081
хэтчбек 3 дв.                747
пикап двойная кабина         551
купе-хардтоп                 228
внедорожник 3 дв.            198
кабриолет                    170
родстер                      152
фургон                       143
микровэн                      46
седан-хардтоп                 19
пикап одинарная кабина        14
седан 2 дв.                    8
лимузин                        8
пикап полуторная кабина        7
внедорожник открытый           6
тарга                          2
фастбек                        1
Name: body_type, dtype: int64

In [304]:
# translate body types:
data.body_type = data.body_type.map(
                {'седан' : 'sedan',
                 'внедорожник 5 дв.' : 'SUV 5 doors.',
                 'лифтбек' : 'liftback',
                 'хэтчбек 5 дв.' : 'hatchback 5 doors.',
                 'универсал 5 дв.' : 'station wagon 5 doors.',
                 'минивэн' : 'minivan',
                 'купе' : 'coupe',
                 'компактвэн' : 'compact MPV',
                 'хэтчбек 3 дв.' : 'hatchback 3 doors.',
                 'пикап двойная кабина' : 'pickup double cab',
                 'купе-хардтоп' : 'coupe-hardtop',
                 'внедорожник 3 дв.' : 'off-road vehicle 3 doors.',
                 'родстер' :  'roadster',
                 'фургон' : 'van',
                 'кабриолет' : 'cabriolet',
                 'седан-хардтоп' : 'hardtop sedan',
                 'микровэн' : 'microvan',
                 'седан 2 дв.' : 'sedan 2 doors.',
                 'лимузин' : 'limousine',
                 'пикап одинарная кабина' : 'pickup single cab',
                 'пикап полуторная кабина' : 'pickup one-and-a-half cab',
                 'внедорожник открытый' :  'off-road vehicle open',
                 'тарга' : 'targa',
                 'фастбек' : 'fastback'})


In [305]:
# sort out:
cat_columns.append("body_type")
cat_columns


['brand', 'model_name', 'body_type']

### 3. doors

In [306]:
data.doors.value_counts()


5    41563
4    25317
2     2060
3     1010
0        1
Name: doors, dtype: int64

In [307]:
# sort out:
cat_columns.append("doors")
cat_columns


['brand', 'model_name', 'body_type', 'doors']

### 4. model_date

In [308]:
data.model_date.describe()


count    69951.000000
mean      2008.272619
std          7.322240
min       1904.000000
25%       2005.000000
50%       2010.000000
75%       2013.000000
max       2021.000000
Name: model_date, dtype: float64

Dataset contains data about car model from year 1904... 

In [309]:
# sort out:
num_columns.append("model_date")
num_columns


['model_date']

In [310]:
# to add year of collecting datasets, find it out: 
# for train and test data it is different:
traindata_year = data[data.test_train=="train"]["model_date"].max()
testdata_year = data[data.test_train=="test"]["model_date"].max()
traindata_year, testdata_year


(2021, 2020)

### ++++++ new column +++++

In [311]:
# create new column "dataset_year" (year of collecting datasets)
data["dataset_year"] = np.where(data.test_train=="test", testdata_year, traindata_year)
data.dataset_year.sample(7)


12308    2021
35862    2020
4563     2021
1652     2021
6542     2021
28909    2021
45290    2020
Name: dataset_year, dtype: int64

In [312]:
# sort out:
bin_columns.append("dataset_year")
bin_columns


['dataset_year']

### ++++++ new column +++++

In [313]:
#### add model age:
data["model_age"]= data.dataset_year - data.model_date
data.model_age.sample(7)


63695    13
35802    12
41443    20
9915     26
44041     4
6935      6
28412     6
Name: model_age, dtype: int64

In [314]:
# sort out:
num_columns.append("model_age")
num_columns


['model_date', 'model_age']

### ++++++ new column +++++

In [315]:
# new column-marker to show if it is a new model on a market:
data["new_model"] = np.where(data.model_age==0, "yes", "no")


In [316]:
data.new_model.value_counts()


no     69927
yes       24
Name: new_model, dtype: int64

In [317]:
# sort out:
bin_columns.append("new_model")
bin_columns


['dataset_year', 'new_model']

### 5. prod_date

In [318]:
data.prod_date.describe()


count    69951.000000
mean      2010.393576
std          7.011943
min       1904.000000
25%       2007.000000
50%       2012.000000
75%       2016.000000
max       2021.000000
Name: prod_date, dtype: float64

In [319]:
# compare statistics for train and test datasets:
[data[data.test_train=="test"].prod_date.describe(), 
 "===============================",
 data[data.test_train=="train"].prod_date.describe()]



[count    34686.000000
 mean      2009.264602
 std          7.047661
 min       1904.000000
 25%       2006.000000
 50%       2011.000000
 75%       2014.000000
 max       2020.000000
 Name: prod_date, dtype: float64,
 '===============================',
 count    35265.000000
 mean      2011.504012
 std          6.796175
 min       1939.000000
 25%       2008.000000
 50%       2013.000000
 75%       2017.000000
 max       2021.000000
 Name: prod_date, dtype: float64]

test contains car produced in 1904, it will be no easy to predict  it's price on the base on train dataset...

In [320]:
# sort out:
num_columns.append("prod_date")
num_columns


['model_date', 'model_age', 'prod_date']

test dataset is one year older. Adding a car age column Add columns "raritet" , "new_model" and make feature categorian by 10 years.....

### ++++++++ new column +++++++++

In [321]:
data["car_age"] = data.dataset_year - data.model_date
data["car_age"].describe()


count    69951.000000
mean        12.231519
std          7.257905
min          0.000000
25%          7.000000
50%         11.000000
75%         15.000000
max        116.000000
Name: car_age, dtype: float64

There are new cars represented in both datasets, oldest car in test is much older then the oldest car in train, will be marked as raritet or even super raritet

In [322]:
# sort out:
num_columns.append("car_age")
num_columns


['model_date', 'model_age', 'prod_date', 'car_age']

In [323]:
# add new column "new_car" for cars with car_age=0:
data["new_car"] = np.where(data["car_age"]==0, "yes", "no")
# add new column "retro" for cars older then 30 years
data["retro"] = np.where(data["car_age"]>30, "yes", "no")
# and new column "raritet" for cars older then 50 years
data["raritet"] = np.where(data["car_age"]>50, "yes", "no")
data[data.car_age>30][["car_age", "retro", "raritet"]].sample(10)


,car_age,retro,raritet
4151,35,yes,no
66088,32,yes,no
6204,35,yes,no
52525,38,yes,no
34702,35,yes,no
66573,32,yes,no
68741,31,yes,no
21757,46,yes,no
6245,34,yes,no
39554,34,yes,no


In [324]:
# sort out:
bin_columns.append("new_car")
bin_columns.append("retro")
bin_columns.append("raritet")
bin_columns


['dataset_year', 'new_model', 'new_car', 'retro', 'raritet']

### 6. color

In [325]:
data.color.value_counts()


чёрный         21181
белый          14064
серый           8830
серебристый     7632
синий           6346
коричневый      3138
красный         3092
зелёный         1626
бежевый         1290
голубой          781
золотистый       490
пурпурный        482
фиолетовый       398
жёлтый           319
оранжевый        258
розовый           24
Name: color, dtype: int64

In [326]:
data.color = data.color.map(
                {"чёрный" : "black",
                "белый" : "white",
                "серый" : "grey",
                "серебристый" : "silver",
                "синий" : "blue",
                "красный" : "red",
                "коричневый" :"braun",
                "зелёный" : "green",
                "бежевый" : "beige",
                "голубой" : "light-blue",
                "золотистый" : "gold",
                "пурпурный" : "purple",
                "фиолетовый" : "violet",
                "жёлтый" : "yellow",
                "оранжевый" : "orange",
                "розовый" : "rose"})

data.color.value_counts()

black         21181
white         14064
grey           8830
silver         7632
blue           6346
braun          3138
red            3092
green          1626
beige          1290
light-blue      781
gold            490
purple          482
violet          398
yellow          319
orange          258
rose             24
Name: color, dtype: int64

In [327]:
# sort out:
cat_columns.append("color")
cat_columns


['brand', 'model_name', 'body_type', 'doors', 'color']

### 7. engine_V

In [328]:
data.engine_V.value_counts()


2.0 LTR    16022
1.6 LTR     9687
3.0 LTR     7501
1.8 LTR     5705
2.5 LTR     4708
2.4 LTR     3833
1.4 LTR     3506
3.5 LTR     3495
1.5 LTR     1957
1.2 LTR     1056
1.3 LTR      914
2.8 LTR      732
4.7 LTR      701
4.5 LTR      701
4.4 LTR      700
4.0 LTR      685
3.2 LTR      554
5.5 LTR      553
2.1 LTR      532
3.7 LTR      528
5.7 LTR      528
5.6 LTR      443
2.2 LTR      432
2.3 LTR      429
2.7 LTR      411
4.2 LTR      364
4.6 LTR      360
3.6 LTR      338
2.9 LTR      332
1.9 LTR      321
5.0 LTR      265
0.7 LTR      210
1.7 LTR      173
3.3 LTR      160
3.1 LTR      136
 LTR         127
4.8 LTR      110
1.0 LTR      105
3.8 LTR      101
4.3 LTR       84
2.6 LTR       80
6.0 LTR       76
3.4 LTR       69
4.1 LTR       54
5.4 LTR       45
6.2 LTR       40
5.2 LTR       23
5.9 LTR       15
6.3 LTR       13
6.6 LTR       13
1.1 LTR        9
5.8 LTR        9
4.9 LTR        4
3.9 LTR        1
5.3 LTR        1
Name: engine_V, dtype: int64

In [329]:
# remove "LTR"
data.engine_V = data.engine_V.apply(lambda x: str(x).replace(" LTR", ""))


In [330]:
# wonder what has happend with value "LTR":
data.engine_V.unique()


array(['1.4', '1.6', '1.2', '2.5', '1.8', '2.0', '1.3', '3.6', '2.8',
       '1.5', '1.9', '3.0', '3.1', '4.0', '2.7', '2.4', '4.2', '2.3',
       '5.0', '6.0', '3.2', '2.1', '', '2.6', '6.3', '5.2', '4.1', '2.9',
       '2.2', '5.9', '3.3', '3.7', '1.7', '3.5', '0.7', '4.4', '4.8',
       '6.6', '4.6', '5.4', '3.4', '5.6', '1.0', '3.8', '4.5', '5.5',
       '4.7', '6.2', '4.3', '5.8', '5.7', '1.1', '4.9', '3.9', '5.3'],
      dtype=object)

There is an empty value instead. Take a look:

In [331]:
data[data.engine_V==""]


,brand,model_name,body_type,doors,model_date,prod_date,color,engine_V,power,fuel,image,mileage,name,config,transmission,owners,pass,drive,wheel,state,custom,car_url,price,test_train,dataset_year,model_age,new_model,car_age,new_car,retro,raritet
3708,AUDI,other,SUV 5 doors.,5,2018,2019,blue,,408 N12,электро,https://avatars.mds.yandex.net/get-autoru-vos/...,28 480 км,AT,ALLROAD_5_DOORS AUTOMATIC,автоматическая,1 владелец,Оригинал,полный,Левый,Не требует ремонта,Растаможен,https://auto.ru/cars/used/sale/audi/e_tron/110...,6 150 000 ₽,train,2021,3,no,3,no,no,no
3931,AUDI,other,SUV 5 doors.,5,2019,2020,blue,,408 N12,электро,https://autoru.naydex.net/w17S7KB96/f11010lkSc...,1 111 км,AT,ALLROAD_5_DOORS AUTOMATIC,автоматическая,1 владелец,Оригинал,полный,Левый,Не требует ремонта,Растаможен,https://auto.ru/cars/used/sale/audi/e_tron_spo...,7 270 000 ₽,train,2021,2,no,2,no,no,no
4075,AUDI,other,SUV 5 doors.,5,2019,2020,silver,,408 N12,электро,https://autoru.naydex.net/w17S7KB96/f11010lkSc...,2 700 км,AT,ALLROAD_5_DOORS AUTOMATIC,автоматическая,1 владелец,Оригинал,полный,Левый,Не требует ремонта,Растаможен,https://auto.ru/cars/used/sale/audi/e_tron_spo...,5 999 999 ₽,train,2021,2,no,2,no,no,no
4161,AUDI,other,SUV 5 doors.,5,2018,2019,light-blue,,408 N12,электро,https://avatars.mds.yandex.net/get-autoru-vos/...,40 000 км,AT,ALLROAD_5_DOORS AUTOMATIC,автоматическая,1 владелец,Оригинал,полный,Левый,Не требует ремонта,Растаможен,https://auto.ru/cars/used/sale/audi/e_tron/110...,6 475 000 ₽,train,2021,3,no,3,no,no,no
4203,AUDI,other,sedan,4,2020,2021,grey,,646 N12,электро,https://autoru.naydex.net/w17S7KB96/f11010lkSc...,250 км,AT,SEDAN AUTOMATIC,автоматическая,1 владелец,Оригинал,полный,Левый,Не требует ремонта,Растаможен,https://auto.ru/cars/used/sale/audi/rs_e_tron_...,17 055 000 ₽,train,2021,1,no,1,no,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67933,NISSAN,LEAF,hatchback 5 doors.,5,2017,2018,black,,150 N12,электро,https://autoru.naydex.net/1xsMU7627/eea2057RFg...,100,Electro AT (111 кВт),HATCHBACK_5_DOORS AUTOMATIC,автоматическая,1 владелец,Оригинал,передний,Левый,Не требует ремонта,Растаможен,https://auto.ru/cars/used/sale/nissan/leaf/110...,0,test,2020,3,no,3,no,no,no
67965,NISSAN,LEAF,hatchback 5 doors.,5,2010,2011,light-blue,,109 N12,электро,https://avatars.mds.yandex.net/get-autoru-vos/...,29500,Electro AT (81 кВт),HATCHBACK_5_DOORS AUTOMATIC,автоматическая,1 владелец,Оригинал,передний,Правый,Не требует ремонта,Растаможен,https://auto.ru/cars/used/sale/nissan/leaf/110...,0,test,2020,10,no,10,no,no,no
68063,NISSAN,LEAF,hatchback 5 doors.,5,2010,2013,black,,109 N12,электро,https://autoru.naydex.net/1xsMU7627/eea2057RFg...,50000,Electro AT (81 кВт),HATCHBACK_5_DOORS AUTOMATIC,автоматическая,1 владелец,Оригинал,передний,Правый,Не требует ремонта,Растаможен,https://auto.ru/cars/used/sale/nissan/leaf/109...,0,test,2020,10,no,10,no,no,no
69424,MERCEDES,B_KLASSE,hatchback 5 doors.,5,2014,2016,white,,179 N12,электро,https://autoru.naydex.net/O1Iap7626/e63034lCq/...,28900,250 e Electro AT (132 кВт),HATCHBACK_5_DOORS AUTOMATIC,автоматическая,2 владельца,Оригинал,передний,Левый,Не требует ремонта,Растаможен,https://auto.ru/cars/used/sale/mercedes/b_klas...,0,test,2020,6,no,6,no,no,no


In [332]:
data.isnull().sum()


brand           0
model_name      0
body_type       0
doors           0
model_date      0
prod_date       0
color           0
engine_V        0
power           0
fuel            0
image           0
mileage         0
name            0
config          0
transmission    0
owners          0
pass            1
drive           0
wheel           0
state           0
custom          0
car_url         0
price           0
test_train      0
dataset_year    0
model_age       0
new_model       0
car_age         0
new_car         0
retro           0
raritet         0
dtype: int64

In [333]:
# sort out:
num_columns.append("engine_V")
num_columns


['model_date', 'model_age', 'prod_date', 'car_age', 'engine_V']

we received empty values for electrocars

In [334]:
### +++++++++++++ new column +++++++++++++++

In [335]:
# create marker column for electrocars:
data["electrocar"] = np.where(data["engine_V"]=="", "yes", "no")
data.electrocar.value_counts()


no     69824
yes      127
Name: electrocar, dtype: int64

In [336]:
# sort out:
bin_columns.append("electrocar")
bin_columns


['dataset_year', 'new_model', 'new_car', 'retro', 'raritet', 'electrocar']

### 8. power

In [337]:
data.power.value_counts()


150 N12    4104
249 N12    4060
110 N12    2557
170 N12    1898
140 N12    1845
           ... 
537 N12       1
187 N12       1
480 N12       1
61 N12        1
301 N12       1
Name: power, Length: 334, dtype: int64

In [338]:
# remove "N12":
data.power = data.power.apply(lambda x: str(x).replace(" N12", "")).astype(int)
data.power.value_counts()


150    4104
249    4060
110    2557
170    1898
140    1845
       ... 
236       1
301       1
430       1
324       1
44        1
Name: power, Length: 334, dtype: int64

In [339]:
# sort out
num_columns.append("power")
num_columns


['model_date', 'model_age', 'prod_date', 'car_age', 'engine_V', 'power']

In [340]:
# check for empty values:
data[data["power"]==""]


,brand,model_name,body_type,doors,model_date,prod_date,color,engine_V,power,fuel,image,mileage,name,config,transmission,owners,pass,drive,wheel,state,custom,car_url,price,test_train,dataset_year,model_age,new_model,car_age,new_car,retro,raritet,electrocar


In [341]:
data.power.sample(7)


18252    190
35762    152
17876    171
15253    184
63238    144
15577    245
43865    100
Name: power, dtype: int64

looks OK now

### 9. fuel

In [342]:
data.fuel.value_counts()


бензин     57409
дизель     11415
гибрид       992
электро      127
газ            8
Name: fuel, dtype: int64

In [343]:
data.fuel = data.fuel.map(
            {
                "бензин" : "petrol",
                "дизель" : "diesel",
                "гибрид" : "hybrid",
                "электро" : "electro",
                "газ" : "gas"
            }
                    )


In [344]:
data.fuel.value_counts()


petrol     57409
diesel     11415
hybrid       992
electro      127
gas            8
Name: fuel, dtype: int64

Too many petrol cars, too little of gas autos. 

### ++++++++++++ new column ++++++++++++

In [345]:
# Add column "petrol_car", as it's the most wanted
data["petrol_car"] = np.where(data.fuel=="petrol", "yes", "no")


In [346]:
# sort out
cat_columns.append("fuel")
bin_columns.append("petrol_car")
[cat_columns, bin_columns, num_columns]


[['brand', 'model_name', 'body_type', 'doors', 'color', 'fuel'],
 ['dataset_year',
  'new_model',
  'new_car',
  'retro',
  'raritet',
  'electrocar',
  'petrol_car'],
 ['model_date', 'model_age', 'prod_date', 'car_age', 'engine_V', 'power']]

### 10. image

In [347]:
data.image.sample(7)


62617    https://autoru.naydex.net/mcQh17801/f1b5751dCi...
53124    https://autoru.naydex.net/mcQh17801/f1b5751dCi...
60384    https://avatars.mds.yandex.net/get-autoru-vos/...
10729    https://avatars.mds.yandex.net/get-autoru-vos/...
41675    https://avatars.mds.yandex.net/get-autoru-vos/...
3641     https://autoru.naydex.net/w17S7KB96/f11010lkSc...
22124    https://avatars.mds.yandex.net/get-autoru-vos/...
Name: image, dtype: object

In [348]:
# let's find out if there are empty or not valid values:
data[data.image.str[:6]!="https:"]


,brand,model_name,body_type,doors,model_date,prod_date,color,engine_V,power,fuel,image,mileage,name,config,transmission,owners,pass,drive,wheel,state,custom,car_url,price,test_train,dataset_year,model_age,new_model,car_age,new_car,retro,raritet,electrocar,petrol_car


In [349]:
data['image_len'] = data.image.astype(str).str.len()


In [350]:
data['image_len'].describe()


count    69951.000000
mean       156.511015
std         63.468004
min         94.000000
25%         94.000000
50%         95.000000
75%        221.000000
max        221.000000
Name: image_len, dtype: float64

In [351]:
data[data.image_len==94]["car_url"].sample()


2674    https://auto.ru/cars/used/sale/skoda/superb/11...
Name: car_url, dtype: object

Looks like all ads have images, the colummn does not contain helpful infornation for feature analysis, will be removed 

### --------- remove column ---------

In [352]:
# remove column:
data.drop(["image"], axis=1, inplace=True)


### 11. mileage

In [353]:
data.mileage.head()


0    182 200 км
1    296 000 км
2    131 807 км
3    192 000 км
4    190 000 км
Name: mileage, dtype: object

In [354]:
data.mileage.sample(10)


10231    300 600 км
7648     334 614 км
33440    110 000 км
66830         53898
34812    115 000 км
25315    222 000 км
62259         17169
28456    235 144 км
25032    106 000 км
1784     178 000 км
Name: mileage, dtype: object

In [355]:
data.mileage[0]


'182\xa0200\xa0км'

data is not clean, remove spaces and letters:

In [356]:
data.mileage = data.mileage.apply(
    lambda x: str(x).replace("\xa0", "").replace("км", "")
    ).astype(int)                                           


In [357]:
data.mileage.describe()


count      69951.000000
mean      153545.958843
std        98243.455772
min            1.000000
25%        82918.500000
50%       141000.000000
75%       205000.000000
max      1000000.000000
Name: mileage, dtype: float64

In [358]:
# sort out:
num_columns.append("mileage")
num_columns


['model_date',
 'model_age',
 'prod_date',
 'car_age',
 'engine_V',
 'power',
 'mileage']

### 12. name

In [359]:
data.name.sample(10)


1761                1.4 MT
35777    1.6 AT (110 л.с.)
34445              2.4 CVT
66130    1.9d MT (68 л.с.)
19617               3.7 AT
30808               1.6 AT
16038               1.8 AT
21470              2.0 CVT
6905                2.4 AT
30006              1.4 AMT
Name: name, dtype: object

In [360]:
len(data.name.value_counts())


2905

In [361]:
data[["engine_V", "transmission", "name", "power"]].sample(10)


,engine_V,transmission,name,power
59363,2.0,роботизированная,Long 2.0d AMT (140 л.с.),140
11212,2.5,автоматическая,2.5 AT,210
2682,1.6,механическая,1.6 MT,102
49666,1.5,механическая,1.5 MT (90 л.с.),90
61238,2.0,автоматическая,2.0d AT (140 л.с.) 4WD,140
50736,2.1,автоматическая,250 d 2.1d AT (204 л.с.) 4WD,204
47483,1.6,механическая,1.6 MT (97 л.с.),97
50807,3.0,автоматическая,400 Long 3.0 AT (333 л.с.) 4WD,333
49072,2.0,механическая,2.0 MT (141 л.с.) 4WD,141
51106,2.9,автоматическая,400 d 2.9d AT (340 л.с.) 4WD,340


Too many different values, column includes engine_V, transmission, power and drive. Can be deleted.

### --------- remove column ---------

In [362]:
data.drop(["name"], axis=1, inplace=True)


### 13. config


In [363]:
data.config.sample(10)


36989      WAGON_5_DOORS AUTOMATIC 1.8
44544    ALLROAD_5_DOORS AUTOMATIC 3.0
35606           LIFTBACK AUTOMATIC 1.6
60527              SEDAN AUTOMATIC 1.8
28241              SEDAN AUTOMATIC 2.5
20166    ALLROAD_5_DOORS AUTOMATIC 3.7
51135              SEDAN AUTOMATIC 3.0
14367              SEDAN AUTOMATIC 2.0
54682     ALLROAD_5_DOORS VARIATOR 1.2
39002              SEDAN AUTOMATIC 4.2
Name: config, dtype: object

contains other columns, will be removed

### --------- remove column ---------

In [364]:
data.drop(["config"], axis=1, inplace=True)


### 14. transmission

In [365]:
data.transmission.value_counts()


автоматическая      39203
механическая        13287
вариатор             9211
роботизированная     8250
Name: transmission, dtype: int64

In [366]:
# map values:
data.transmission = data.transmission.map(
            {"автоматическая" : "AT",
            "механическая" : "MT",
            "вариатор" : "CVT",
            "роботизированная" : "AMT"}
)

data.transmission.value_counts()


AT     39203
MT     13287
CVT     9211
AMT     8250
Name: transmission, dtype: int64

In [367]:
# sort out:
cat_columns.append("transmission")
cat_columns


['brand', 'model_name', 'body_type', 'doors', 'color', 'fuel', 'transmission']

### 15. owners

In [368]:
data.owners.value_counts()


3 или более    31221
1 владелец     21146
2 владельца    17584
Name: owners, dtype: int64

In [369]:
data.owners.value_counts().keys()[0],\
data.owners.value_counts().keys()[1],\
data.owners.value_counts().keys()[2]


('3 или более', '1\xa0владелец', '2\xa0владельца')

Column contains 3 values. As 1 owner is super good, add a flag to a car record: column "1_owner" 

In [370]:
dict_owners = {"3 или более":3, "1\xa0владелец": 1, "2\xa0владельца":2}
data.owners = data.owners.map(dict_owners)
data.owners.value_counts()


3    31221
1    21146
2    17584
Name: owners, dtype: int64

In [371]:
data["1_owner"] = np.where(data.owners==1, "yes", "no")


In [372]:
# sort out:
cat_columns.append("owners")
bin_columns.append("1_owner")
[cat_columns, bin_columns]


[['brand',
  'model_name',
  'body_type',
  'doors',
  'color',
  'fuel',
  'transmission',
  'owners'],
 ['dataset_year',
  'new_model',
  'new_car',
  'retro',
  'raritet',
  'electrocar',
  'petrol_car',
  '1_owner']]

### 16. pass


In [373]:
data["pass"].value_counts()


Оригинал    60310
Дубликат     9640
Name: pass, dtype: int64

column contains 2 values, convert it into a flag: 1 for original passport

In [374]:
data["pass"] = data["pass"].map({"Оригинал":"Original", "Дубликат":"Duplicate"})
data["pass"].value_counts()


Original     60310
Duplicate     9640
Name: pass, dtype: int64

In [375]:
# sort out:
bin_columns.append("pass")
bin_columns


['dataset_year',
 'new_model',
 'new_car',
 'retro',
 'raritet',
 'electrocar',
 'petrol_car',
 '1_owner',
 'pass']

### 17. drive

In [376]:
data["drive"].value_counts()


полный      32569
передний    30711
задний       6671
Name: drive, dtype: int64

not much of rare drive cars... not suitable for winter...  process as category feature:

In [377]:
data["drive"] = data["drive"].map({"передний":"FWD", "полный":"4WD", "задний":"RWD"})
data["drive"].value_counts()


4WD    32569
FWD    30711
RWD     6671
Name: drive, dtype: int64

In [378]:
cat_columns.append("drive")
cat_columns


['brand',
 'model_name',
 'body_type',
 'doors',
 'color',
 'fuel',
 'transmission',
 'owners',
 'drive']

### 18. wheel

In [379]:
data.wheel.value_counts()


Левый     65907
Правый     4044
Name: wheel, dtype: int64

binary feature, left steeering wheel will map to 1:

In [380]:
data["wheel"] = data["wheel"].map({"Левый": "Left", "Правый":"Right"})
data["wheel"].value_counts()


Left     65907
Right     4044
Name: wheel, dtype: int64

In [381]:
bin_columns.append("wheel")
bin_columns


['dataset_year',
 'new_model',
 'new_car',
 'retro',
 'raritet',
 'electrocar',
 'petrol_car',
 '1_owner',
 'pass',
 'wheel']

### 19. state

In [382]:
data.state.value_counts()


Не требует ремонта    69947
Битый / не на ходу        4
Name: state, dtype: int64

binary feature, the second value is too rare, but really important, it can 10 times reduce the price

### ------------------ remove column -------------------

In [383]:
# negative value for this column will significantly reduce price,
# but it is too rare be counted:
data.drop("state", axis=1, inplace=True)


### 20. custom

In [384]:
data.custom.value_counts()


Растаможен       69950
Не растаможен        1
Name: custom, dtype: int64

### ------------------ remove column -------------------

In [385]:
# negative value for this column will significantly reduce price,
# but it is too rare be counted:
data.drop("custom", axis=1, inplace=True)


### 21. car_url

not neede for analysis, delete:

In [386]:
data.drop("car_url", axis=1, inplace=True)


### 22. price

In [387]:
data.price.sample(10)


49740              0
39827              0
55748              0
58194              0
17379      240 000 ₽
17933    1 084 000 ₽
69416              0
64713              0
36383              0
21057      370 000 ₽
Name: price, dtype: object

contains 0-values, hopefully all from test dataset, let's take a look:

In [388]:
len(data[(data.price==0 )& (data.test_train=="train")]), len(df_test)


(0, 34686)

Train dataset doesn't contain 0 as a target value - OK

In [389]:
# take a deeper look at values:
data.price[0], data.price[1], data.price[3]


('370\xa0000\xa0₽', '155\xa0000\xa0₽', '470\xa0000\xa0₽')

Ooooopsss! Needs to be corrected

In [390]:
data.price = data.price.astype(str)
data.price = data["price"].apply(lambda x: x.replace("\xa0", "").replace("₽", ""))   
data.price = data.price.astype(int)
data.price.sample(10)


29917    1130000
35801          0
49242          0
23552    1478000
65936          0
57119          0
2930      538000
51329          0
7088      369000
4874     2397000
Name: price, dtype: int64

In [391]:
data[data.price==0].test_train.value_counts()


test    34686
Name: test_train, dtype: int64

Price column looks OK now

### !!! For this column, will try to apply scaling, to take inflation into account

validate column lists:


In [392]:
all_columns = list(data.columns)
num_columns = list(set(num_columns))
cat_columns = list(set(cat_columns))
bin_columns = list(set(bin_columns))


In [393]:
print("num_columns:", num_columns)
print("cat_columns:", cat_columns)
print("bin_columns:", bin_columns)


num_columns: ['car_age', 'power', 'engine_V', 'model_date', 'model_age', 'mileage', 'prod_date']
cat_columns: ['body_type', 'fuel', 'model_name', 'brand', 'drive', 'doors', 'color', 'owners', 'transmission']
bin_columns: ['new_car', 'new_model', 'dataset_year', 'pass', 'retro', '1_owner', 'wheel', 'electrocar', 'raritet', 'petrol_car']


In [394]:
(set(num_columns) | set(cat_columns) | set(bin_columns)) - set(all_columns)


set()

In [395]:
set(all_columns) - (set(num_columns) | set(cat_columns) | set(bin_columns))


{'image_len', 'price', 'test_train'}

In [396]:
# we need price and test_train, remove image_len:
data.drop("image_len", axis=1, inplace=True)
# Columns "custom" and "state" "Not OK" values will significantly 
# reduce the price But, as not_OK values are too rare, columns will be removed




## SAVE DATA TO FILES:

In [397]:
# make sure data is OK to be saved:
data.head(7)#sample(7).T


,brand,model_name,body_type,doors,model_date,prod_date,color,engine_V,power,fuel,mileage,transmission,owners,pass,drive,wheel,price,test_train,dataset_year,model_age,new_model,car_age,new_car,retro,raritet,electrocar,petrol_car,1_owner
0,SKODA,Skoda Fabia II,station wagon 5 doors.,5,2007,2009,black,1.4,86,petrol,182200,MT,2,Original,FWD,Left,370000,train,2021,14,no,14,no,no,no,no,yes,no
1,SKODA,Skoda Fabia I,station wagon 5 doors.,5,1999,2001,silver,1.4,68,petrol,296000,MT,3,Duplicate,FWD,Left,155000,train,2021,22,no,22,no,no,no,no,yes,no
2,SKODA,Skoda Octavia III (A7) Рестайлинг,liftback,5,2017,2017,white,1.6,110,petrol,131807,AT,1,Original,FWD,Left,840000,train,2021,4,no,4,no,no,no,no,yes,yes
3,SKODA,other,hatchback 5 doors.,5,2010,2012,blue,1.4,180,petrol,192000,AMT,3,Original,FWD,Left,470000,train,2021,11,no,11,no,no,no,no,yes,no
4,SKODA,Skoda Fabia II Рестайлинг,hatchback 5 doors.,5,2010,2011,black,1.2,70,petrol,190000,MT,2,Original,FWD,Left,290000,train,2021,11,no,11,no,no,no,no,yes,no
5,SKODA,Skoda Octavia I (A4) Рестайлинг,station wagon 5 doors.,5,2000,2007,silver,1.4,75,petrol,198500,MT,2,Original,FWD,Left,225000,train,2021,21,no,21,no,no,no,no,yes,no
6,SKODA,other,sedan,4,2001,2002,black,2.5,155,diesel,252000,AT,3,Original,FWD,Left,275000,train,2021,20,no,20,no,no,no,no,no,no


In [398]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69951 entries, 0 to 69950
Data columns (total 28 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         69951 non-null  object
 1   model_name    69951 non-null  object
 2   body_type     69951 non-null  object
 3   doors         69951 non-null  int64 
 4   model_date    69951 non-null  int64 
 5   prod_date     69951 non-null  int64 
 6   color         69951 non-null  object
 7   engine_V      69951 non-null  object
 8   power         69951 non-null  int64 
 9   fuel          69951 non-null  object
 10  mileage       69951 non-null  int64 
 11  transmission  69951 non-null  object
 12  owners        69951 non-null  int64 
 13  pass          69950 non-null  object
 14  drive         69951 non-null  object
 15  wheel         69951 non-null  object
 16  price         69951 non-null  int64 
 17  test_train    69951 non-null  object
 18  dataset_year  69951 non-null  int64 
 19  mode

In [399]:
data.to_csv(path + "PROCESSED/data.csv", index=False)


In [400]:
df_num_columns = pd.DataFrame(num_columns)
df_cat_columns = pd.DataFrame(cat_columns)
df_bin_columns = pd.DataFrame(bin_columns)


df_num_columns.to_csv(path + "PROCESSED/num_columns.csv", mode='w', header=False, index=False)
df_cat_columns.to_csv(path + "PROCESSED/cat_columns.csv", mode='w', header=False, index=False)
df_bin_columns.to_csv(path + "PROCESSED/bin_columns.csv", mode='w', header=False, index=False)


In [401]:
# quick check:
data = pd.read_csv(path + "PROCESSED/data.csv")
data.head(7)#.T


,brand,model_name,body_type,doors,model_date,prod_date,color,engine_V,power,fuel,mileage,transmission,owners,pass,drive,wheel,price,test_train,dataset_year,model_age,new_model,car_age,new_car,retro,raritet,electrocar,petrol_car,1_owner
0,SKODA,Skoda Fabia II,station wagon 5 doors.,5,2007,2009,black,1.4,86,petrol,182200,MT,2,Original,FWD,Left,370000,train,2021,14,no,14,no,no,no,no,yes,no
1,SKODA,Skoda Fabia I,station wagon 5 doors.,5,1999,2001,silver,1.4,68,petrol,296000,MT,3,Duplicate,FWD,Left,155000,train,2021,22,no,22,no,no,no,no,yes,no
2,SKODA,Skoda Octavia III (A7) Рестайлинг,liftback,5,2017,2017,white,1.6,110,petrol,131807,AT,1,Original,FWD,Left,840000,train,2021,4,no,4,no,no,no,no,yes,yes
3,SKODA,other,hatchback 5 doors.,5,2010,2012,blue,1.4,180,petrol,192000,AMT,3,Original,FWD,Left,470000,train,2021,11,no,11,no,no,no,no,yes,no
4,SKODA,Skoda Fabia II Рестайлинг,hatchback 5 doors.,5,2010,2011,black,1.2,70,petrol,190000,MT,2,Original,FWD,Left,290000,train,2021,11,no,11,no,no,no,no,yes,no
5,SKODA,Skoda Octavia I (A4) Рестайлинг,station wagon 5 doors.,5,2000,2007,silver,1.4,75,petrol,198500,MT,2,Original,FWD,Left,225000,train,2021,21,no,21,no,no,no,no,yes,no
6,SKODA,other,sedan,4,2001,2002,black,2.5,155,diesel,252000,AT,3,Original,FWD,Left,275000,train,2021,20,no,20,no,no,no,no,no,no


In [402]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69951 entries, 0 to 69950
Data columns (total 28 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   brand         69951 non-null  object 
 1   model_name    69951 non-null  object 
 2   body_type     69951 non-null  object 
 3   doors         69951 non-null  int64  
 4   model_date    69951 non-null  int64  
 5   prod_date     69951 non-null  int64  
 6   color         69951 non-null  object 
 7   engine_V      69824 non-null  float64
 8   power         69951 non-null  int64  
 9   fuel          69951 non-null  object 
 10  mileage       69951 non-null  int64  
 11  transmission  69951 non-null  object 
 12  owners        69951 non-null  int64  
 13  pass          69950 non-null  object 
 14  drive         69951 non-null  object 
 15  wheel         69951 non-null  object 
 16  price         69951 non-null  int64  
 17  test_train    69951 non-null  object 
 18  dataset_year  69951 non-nu

Looks OK for future processing

### Please continue with 3. EDA

In [403]:
print("done")

done
